In [50]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [51]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
from datetime import datetime
from pathlib import Path

In [52]:
path = "/home/edoardo/Desktop/University/KU Leuven/Advancedanalytics/ThirdAssignment/output"

In [53]:
def save_only_vandals(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df = df.filter(df['label'] == 'vandal')
    dt = datetime.now().strftime("%m-%d-%Y,%H:%M:%S")
    directory = f"{path}/vandals/{dt}"
    df = df.toPandas()
    if len(df) != 0:
        Path(directory).mkdir(parents=True, exist_ok=True)
        df.to_csv(f"{directory}/df.csv", sep="|")

In [54]:
def save_all(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    dt = datetime.now().strftime("%m-%d-%Y,%H:%M:%S")
    directory = f"{path}/mixed/{dt}"
    df = df.toPandas()
    if len(df) != 0:
        Path(directory).mkdir(parents=True, exist_ok=True)
        df.to_csv(f"{directory}/df.csv", sep="|")

In [55]:
ssc = StreamingContext(sc, 10)

In [56]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(save_only_vandals)

In [57]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [58]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
